Importing Dependencies

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

Device Configuration

In [4]:
# Check for CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")

using device: cuda


Data Collection and Preprocessing

In [5]:
# Load the breast cancer dataset
data = load_breast_cancer()
X, y = data.data, data.target

In [6]:
# Splitting the data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [7]:
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (455, 30)
y_train shape: (455,)


In [8]:
# Standardizing our data using standard scalar -  this is not requireds using random forests/xg boost
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

type(X_train)


numpy.ndarray

In [9]:
# convert the data to pytorch tensors and move it to the GPU
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
# do the same for the testing data
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

Creating our Neural Network

In [10]:
# Define the neural Network Architecture

class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, output_size):
    super(NeuralNet, self).__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.fc2 = nn.Linear(hidden_size, output_size)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.fc2(out)
    out = self.sigmoid(out)
    return out


In [11]:
# define hyperparameters and all parameters we are going to need
input_size = X_train.shape[1]
hidden_size = 64
output_size = 1
learning_rate = 0.001
num_epochs = 100

In [12]:
# initialize the neural networkand move it to the GPU
model = NeuralNet(input_size, hidden_size, output_size).to(device)

In [13]:
# define the loss and the optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

Training the Neural Network

In [14]:
# Training the model
for epoch in range(num_epochs):
  # forward pass and loss calculation
  model.train()
  optimizer.zero_grad()
  outputs = model(X_train)
  loss = criterion(outputs, y_train.view(-1, 1))

  # backward pass and optimization
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  # calculate accuracy of model
  with torch.no_grad():
    predicted = outputs.round()
    correct = (predicted == y_train.view(-1, 1)).float().sum().item()
    accuracy = correct / y_train.size(0)

  if (epoch+1) % 10 == 0:
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

Epoch [10/100], Loss: 0.5560, Accuracy: 0.8725
Epoch [20/100], Loss: 0.4237, Accuracy: 0.9319
Epoch [30/100], Loss: 0.3299, Accuracy: 0.9363
Epoch [40/100], Loss: 0.2642, Accuracy: 0.9473
Epoch [50/100], Loss: 0.2179, Accuracy: 0.9560
Epoch [60/100], Loss: 0.1846, Accuracy: 0.9626
Epoch [70/100], Loss: 0.1600, Accuracy: 0.9626
Epoch [80/100], Loss: 0.1415, Accuracy: 0.9670
Epoch [90/100], Loss: 0.1272, Accuracy: 0.9758
Epoch [100/100], Loss: 0.1158, Accuracy: 0.9758


Model Evaluation

In [15]:
# Evaluatiion on training set, to ensure that it isnnot overfitted or underfitted
model.eval()
with torch.no_grad():
  outputs = model(X_train)
  predicted = outputs.round()
  correct = (predicted == y_train.view(-1, 1)).float().sum().item()
  accuracy = correct / y_train.size(0)
  print(f"Training Accuracy: {accuracy:.4f}")

Training Accuracy: 0.9758


In [16]:
# Evaluation on testing data
model.eval()
with torch.no_grad():
  outputs = model(X_test)
  predicted = outputs.round()
  correct = (predicted == y_test.view(-1, 1)).float().sum().item()
  accuracy = correct / y_test.size(0)
  print(f"Testing Accuracy: {accuracy:.4f}")

Training Accuracy: 0.9737
